In [1]:
import os


symbol = 'BTCSTUSDT'
time_interval = '1m'

In [ ]:
from datetime import timezone
from pathlib import Path

import polars as pl
from dateutil import parser
from datetime import timedelta

from config import DataFrequency, BINANCE_DATA_DIR, TradeType

from util.ts_manager import TSManager
from aws.kline.parse import read_csv
from aws.client_async import AwsKlineClient
from merge.kline import merge_klines

pl.Config.set_tbl_rows(30)


In [ ]:
symbols = (BINANCE_DATA_DIR / "parsed_data" / TradeType.spot.value / "klines").glob("*")
symbols = sorted(s.name for s in symbols)

dfs = []
for symbol in symbols:
    df_gap = scan_gaps(TradeType.spot, symbol, time_interval, timedelta(days=1), 0)
    if not df_gap.is_empty():
        print(symbol)
        dfs.append(df_gap.with_columns(pl.lit(symbol).alias("symbol")))

pl.concat(dfs)

In [ ]:
symbols = (BINANCE_DATA_DIR / "parsed_data" / TradeType.um_futures.value / "klines").glob("*")
symbols = sorted(s.name for s in symbols)

dfs = []
for symbol in symbols:
    df_gap = scan_gaps(TradeType.um_futures, symbol, time_interval, timedelta(days=1), 0.1)
    if not df_gap.is_empty():
        print(symbol)
        dfs.append(df_gap.with_columns(pl.lit(symbol).alias("symbol")))

pl.concat(dfs)

In [ ]:
df = merge_klines(TradeType.um_futures, 'ICPUSDT', '1m')
display(df.filter(pl.col('candle_begin_time') < parser.parse("2022-09-27 00:00:00+00:00")).tail())
display(df.filter(pl.col('candle_begin_time') > parser.parse("2022-09-27 00:00:00+00:00")).head())

In [ ]:
symbol_kline_dir = BINANCE_DATA_DIR / 'parsed_data' / TradeType.spot.value / 'klines' / symbol / time_interval

mgr = TSManager(symbol_kline_dir)
c = mgr.get_row_count_per_date()
c.filter(pl.col('row_count') != 1440)

df1 = mgr.read_all()
df1

In [ ]:
api_dir = Path('/home/lostleaf/crypto_data/binance_data/api_data/kline/spot/BTCUSDT/1m')
api_files = sorted(api_dir.glob('*.pqt'))
ldf2 = pl.scan_parquet(api_files).drop('candle_end_time')

ldf = pl.concat([ldf1, ldf2])
ldf = ldf.unique('candle_begin_time', keep='last').sort('candle_begin_time')
ldf = ldf.with_columns(
    (pl.col('quote_volume') / pl.col('volume'))
    .fill_null(pl.col('open'))
    .clip(pl.col('low'), pl.col('high'))
    .alias('vwap1m')
)

df = ldf.collect()
df.tail(10)